## Семинар 1

In [1]:
False == False in [False]

True

In [3]:
for i in range(3):
    print(i)

print(i)

0
1
2
2


Объекты выбают изменяемыми и неизменяемыми
Переменная - это ссылка на объект

In [4]:
lst = "abcd"
for i, sym in enumerate(lst):
    print(i, sym)

0 a
1 b
2 c
3 d


In [5]:
lst2 = list(range(4))
for el1, el2 in zip(lst, lst2):
    print(el1, el2)

a 0
b 1
c 2
d 3


In [10]:
from collections import defaultdict, Counter
a = Counter()
a['test'] += 1
a

Counter({'test': 1})

In [11]:
b = defaultdict(str)
b['test'] += 'hi'
b

defaultdict(str, {'test': 'hi'})

In [12]:
def f(*args, **kwargs):
    print(args)
    print(kwargs)

f(1, 2, a = 5, b = 10)

(1, 2)
{'a': 5, 'b': 10}


## Семинар 2

In [1]:
import sys
import chardet

In [5]:
beer = " W привет some"
[sys.getsizeof(beer[:index]) for index in range(len(beer) + 1)]

[49, 50, 51, 52, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102]

чем больше разных символов используется в строке, тем больше весит строка

In [7]:
chardet.detect(beer.encode())

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [10]:
byte_str = "легко".encode("utf-8")
print(byte_str)
print(byte_str.decode("utf-8"))

b'\xd0\xbb\xd0\xb5\xd0\xb3\xd0\xba\xd0\xbe'
легко


## Семинар 3 (декораторы)

## Семинар 4 (генераторы)

In [2]:
n = 9
m = 3

In [52]:
exclude = [3, 5, 7]
list(i for i in filter(lambda x: x not in exclude, range(10)))

[0, 1, 2, 4, 6, 8, 9]

In [3]:
list(map(lambda x: int(x * m + m * (m - 1) / 2), filter(lambda x: not x % m, range(n))))

[3, 12, 21]

In [62]:
it = iter(range(n))
# list(map(sum, zip(*[it] * m)))
list(zip(*[it] * m))

[(0, 1, 2), (3, 4, 5), (6, 7, 8)]

In [53]:
it = iter(range(4))

list(zip(*[iter(range(4))] * 2))

[(0, 1), (2, 3)]

In [66]:
def chain(*iterables):
    for iterable in iterables:
        for it in iterable:
            yield it
list(chain(range(5), [10, 20], "test"))

[0, 1, 2, 3, 4, 10, 20, 't', 'e', 's', 't']

In [76]:
# def echo(value=None):
#     print("Execution starts when 'next()' is called for the first time.")
#     try:
#         while True:
#             try:
#                 value = (yield value)
#             except Exception as e:
#                 value = e
#     finally:
#         print("Don't forget to clean up when 'close()' is called.")

def echo(value=None):
    while True:
        try:
            value = (yield value)
        except Exception as e:
            value = e

generator = echo(1)
print(next(generator))

print(next(generator))

print(generator.send(2))

generator.throw(TypeError, "spam")

generator.close()

1
None
2


In [77]:
def coroutine():
    for i in range(1, 10):
        print("From generator {}".format((yield i)))
c = coroutine()
c.send(None)
try:
    while True:
        print("From user {}".format(c.send(1)))
except StopIteration: pass

From generator 1
From user 2
From generator 1
From user 3
From generator 1
From user 4
From generator 1
From user 5
From generator 1
From user 6
From generator 1
From user 7
From generator 1
From user 8
From generator 1
From user 9
From generator 1


In [132]:
def helpless():
    for i in range(10):
        try:
            x = yield i
        except Exception as e:
            x = e
            
        print('x', x)
        
c = helpless()
next(c)
print(c.send(10))
print(c.send(10))
c.throw(TypeError('spam'))
print(c.send(10))
c.close()
try:
    print(c.send(5))
except Exception as e:
    print('fail')

x 10
1
x 10
2
x spam
x 10
4
fail


## Семинар 5 (Дескрипторы и метаклассы)

In [ ]:
class Limited(object):

    def __init__(self, lo, hi, label):
        self.lo = lo
        self.hi = hi
        self.label = label
        self.__val = None

    def get_val(self):
        return self.__val

    def set_val(self, value):
        if not (self.lo < value < self.hi):
            raise ValueError("Please, set {label} with conditions: {lo} < {label} < {hi}".format(**self.__dict__))
        self.__val = value

    def del_val(self):
        self.__val = 'No more'

    val = property(get_val, set_val, del_val, "Это свойство val.")

In [2]:
class Mine(object):
    def __init__(self):
        self._x = None

    x = property()

    @x.getter
    def x(self):
        """Это свойство x."""
        return self._x

    @x.setter
    def x(self, value):
        self._x = value

    @x.deleter
    def x(self):
        self._x = 'No more'

In [3]:
a, b = Mine(), Mine()
a.x = 5
print(a.x)
print(b.x)
del(a.x)
print(a.x)

5
None
No more


In [4]:
class Limited(object):

    def __init__(self, lo, hi, label=None):
        self.lo = lo
        self.hi = hi
        self.label = label
        self.__val = None
    
    @property
    def val(self):
        "Это свойство val."
        return self.__val

    @val.setter
    def val(self, value):
        if not (self.lo < value < self.hi):
            raise ValueError("Please, set {label} with conditions: {lo} < {label} < {hi}".format(**self.__dict__))
        self.__val = value
    
    @val.deleter
    def val(self):
        self.__val = 'No more'


class Summator(object):
    x = Limited(1, 1000)
    y = Limited(0, 1)

summator = Summator()
summator.x = 1e2
print(summator.x)

100.0


In [5]:
class Trace(object):

    def __set__(self, *args):
        print('__set__', args, sep="\n")

    def __get__(self, *args):
        print(self, '__get__', args, sep="\n")

    def __delete__(self, *args):
        print('__delete__', args, sep="\n")

class Test(object):
    attr = Trace()

t = Test()
t.attr = 5
print(t.attr)
# del t.attr

__set__
(<__main__.Test object at 0x7f6a01427be0>, 5)
__get__
(<__main__.Test object at 0x7f6a01427be0>, <class '__main__.Test'>)
None


In [3]:
type(int)
try:
    1 + 1
    raise RuntimeError
except ZeroDivisionError:
    pass

RuntimeError: 

In [25]:
class NonNegative:
    
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Cannot be negative.')
        instance.__dict__[self.name] = value
        
    def __set_name__(self, owner, name):
        self.name = name
        
class Order:
    
    price = NonNegative()
    quantity = NonNegative()
    
    def __init__(self, name, price, quantity):
        self._name = name
        self.price = price
        self.quantity = quantity
        
    def total(self):
        return self.price * self.quantity

In [26]:
apple_order = Order('apple', 1, 10)
new_order = Order('apple_pie', 2, 3)
print(new_order.total())
print(apple_order.total())
new_order.__dict__
# 10
# apple_order.price = -10
# # ValueError: Cannot be negative
# apple_order.quantity = -10
# # ValueError: Cannot be negative

6
10


{'_name': 'apple_pie', 'price': 2, 'quantity': 3}

In [45]:
def my_metaclass(a, b, c):
    return a

class C(metaclass=my_metaclass):
    pass

print(C)
B = type('MyClass', (object, ), {'hui': '7sm'})
b = B()
print(b.hui)

C
7sm


In [1]:
class UpperAttrMetaclass(type): 

    def __new__(cls, name, bases, dct):

        attrs = ((name, value) for name, value in dct.items() if not name.startswith('__'))
        uppercase_attr = dict((name.upper(), value) for name, value in attrs)

        return type.__new__(cls, name, bases, uppercase_attr)

In [2]:

class Foo(metaclass=UpperAttrMetaclass): 
    bar = 'bip'

f = Foo()
print(hasattr(Foo, 'bar'))
print(hasattr(Foo, 'BAR'))
print(f.BAR)

False
True
bip


In [36]:
class JsonCreator(type):
    def __new__(mcls, name):
        val = 5 if name == 'doodo' else 3
        return type.__new__(mcls, name, (), {'a': val})
    
cls = JsonCreator('xuo')
cls2 = JsonCreator('doodo')
obj, obj2 = cls(), cls2()
print(obj.a, obj2.a)

3 5


In [8]:
s = 'def f(a):\n        return a + 5'
exec(s)
f(5)

10

In [38]:
a = tuple(i for i in range(5))
a

(0, 1, 2, 3, 4)

In [22]:
f_text = 'def f(self, a):\n        return a + 5'
class S(object):
    exec(f_text)
    
    def __init__(self):
        
        self.f = f
        
obj = S()
obj.f(66)

71

In [27]:
class D:
    def __init__(self):
        self.__dict__['x'] = 5
        
    def module(self):
        def test():
            print(5)

        method_to_call = locals()['test']
        method_to_call()
        
d = D()
d.x
d.module()

5


In [28]:
def f(obj):
    print('attr =', obj.attr)

Foo = type(
    'Foo',
    (),
    {
        'attr': 100,
        'attr_val': f
    }
)

In [29]:
import inspect
inspect.getsource(f)

"def f(obj):\n    print('attr =', obj.attr)\n"

In [35]:
import json

obj = Foo()
obj.attr
json.dumps(obj.__dict__)

'{}'

In [49]:
class A:
    x = 5
    
    def f(self):
        return 5

A.__dict__

mappingproxy({'__module__': '__main__',
              'x': 5,
              'f': <function __main__.A.f(self)>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [53]:
s = "    sdstrins sdf asd  \n"
print(s)
print(s.strip())

    sdstrins sdf asd  

sdstrins sdf asd


# Python & WEB

## "Обычные запросы" в google

In [46]:
from requests import get, post
import urllib
import urllib.parse
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML

In [53]:
def google(query):
    template = 'https://www.google.ru/search?{}'
    url = template.format(urllib.parse.urlencode({'q' : query}))
    req = get(url)
    
    assert req.status_code == 200, 'request failed'
    soup = BeautifulSoup(req.text)
    
    for i, li in enumerate(soup.findAll('h3', attrs={'class' : 'r'})):
        display(HTML('<h2>Operating systems</h2>'))
        link = li.find('a')
        if link:
            link['href'] = 'http://www.google.com' +  link['href']
            link = '<i>{}.</i>  {}'.format(i, link)
            display(HTML(str(link)))

            
google('Hello, World!')

In [59]:
req = get('https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D0%BB%D0%BE%D1%81%D0%BE%D1%84%D0%B8%D1%8F')
# display(HTML(req.text))

soup = BeautifulSoup(req.text)
content = soup.findAll('div', attrs={'id': 'content'})
for a in content:
    p = a.findAll('p')
    display(HTML(str(p)))

In [12]:
def googleSearch(query):
    with requests.session() as c:
        url = 'https://www.google.co.in'
        query = {'q': query}
        urllink = requests.get(url, params=query)
        print(urllink.url)

googleSearch('Linkin Park')

https://www.google.co.in/?q=Linkin+Park


In [43]:
query = ""
template = 'http://localhost:80/'
url = template.format(urllib.parse.urlencode({'q' : query}))
req = get(url)

assert req.status_code == 200, 'request failed'
soup = BeautifulSoup(req.text, "lxml")

display(HTML(str(soup.findAll("table", attrs={"class": "images_table"})[0])))

IndexError: list index out of range

In [45]:
def login(username, password):
    req = post("http://localhost:80/login", data={"username": username, "password": password})
    display(HTML(str(req.text)))
    
login("holyfame", "pass")